<a href="https://colab.research.google.com/github/Sushmithaa-Pandian/NLP/blob/main/Copy_of_indic_main_adj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install tensorflow tensorflow_hub spektral
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 17.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
from spektral.layers import GCNConv, GlobalSumPool, GraphSageConv
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow_text as text
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [5]:
df=pd.read_csv('/content/gdrive/MyDrive/Models_Sushmithaa/indicpophash.csv',nrows=1000)

In [6]:
df

,Unnamed: 0,id,conversation_id,created_at,date,time,time_zone,user_id,username,name,...,likes_count,hashtags,link,retweet,geo,language,pop_score,label,label_new,pop_score_log
0,0,1.511390e+18,1.511390e+18,2022-04-05 17:05:33,2022-04-05,17:05:33,UTC,9.659076e+08,ganeshhirennava,Ganesh H,...,0.0,"['ಅನಕ್ಷರಸ್ಥರ_ಪಂಚರ್_ಶಾಪ್', 'ಮೊಂಡ್_ಹಲಾಲ್_ಸರ್ಟಿಫಿ...",http://twitter.com/anyuser/status/151138959227...,False,NaN,kn,2.0,2.0,2.0,0.693147
1,1,1.509950e+18,1.509950e+18,2022-04-01 17:45:50,2022-04-01,17:45:50,UTC,1.395262e+09,LaxmiputraDesai,DESAI LAXMIPUTRA,...,0.0,['BoycottHimalaya'],http://twitter.com/anyuser/status/150995017899...,False,NaN,kn,42.0,3.0,4.0,3.737670
2,2,1.509589e+18,1.509589e+18,2022-03-31 17:52:11,2022-03-31,17:52:11,UTC,1.468286e+18,hops_neil,Wilson Morgan ⚡⚡⚡⚡🌩️🌩️🌩️,...,0.0,['BoycottHimalaya'],http://twitter.com/anyuser/status/150958938914...,False,NaN,kn,1.0,1.0,2.0,0.000000
3,3,1.510501e+18,1.510501e+18,2022-04-03 06:12:39,2022-04-03,06:12:39,UTC,1.328367e+18,repalli_siva,#Hyderabadi,...,0.0,['BoycottHimalaya'],http://twitter.com/anyuser/status/151050050887...,False,NaN,te,4.0,2.0,3.0,1.386294
4,4,1.513160e+18,1.513160e+18,2022-04-10 14:20:31,2022-04-10,14:20:31,UTC,1.489971e+18,PrashanthSkp1,Prashanth Skp,...,0.0,"['brand', 'boss']",http://twitter.com/anyuser/status/151315999909...,False,NaN,kn,6.0,3.0,3.0,1.791759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,1.508698e+18,1.508698e+18,2022-03-29 06:50:20,2022-03-29,06:50:20,UTC,1.308756e+18,BauriRamprasad,Ramprasad Bauri,...,0.0,"['westbengal', 'aitc']",http://twitter.com/anyuser/status/150869805096...,False,NaN,bn,105.0,3.0,4.0,4.653960
996,996,1.508476e+18,1.508476e+18,2022-03-28 16:09:18,2022-03-28,16:09:18,UTC,1.133900e+18,mindfuldesserts,Edchat,...,0.0,"['assam', 'education']",http://twitter.com/anyuser/status/150847633245...,False,NaN,bn,1.0,1.0,2.0,0.000000
997,997,1.508457e+18,1.508457e+18,2022-03-28 14:52:03,2022-03-28,14:52:03,UTC,1.477211e+18,D_Punorutthan,Daily Punorutthan©,...,1.0,"['DailyPunorutthan', 'India', 'BreakingNews', ...",http://twitter.com/anyuser/status/150845689174...,False,NaN,bn,1.0,1.0,2.0,0.000000
998,998,1.508401e+18,1.508401e+18,2022-03-28 11:11:50,2022-03-28,11:11:50,UTC,1.477211e+18,D_Punorutthan,Daily Punorutthan©,...,1.0,"['DailyPunorutthan', 'DU', 'BreakingNews', 'ঢা...",http://twitter.com/anyuser/status/150840147472...,False,NaN,bn,1.0,1.0,2.0,0.000000


In [7]:
from sklearn.preprocessing import minmax_scale
import pandas as pd

df["class"]= minmax_scale(df["pop_score_log"])

y=df["class"].mean()
df['category']=df['class'].apply(lambda x: 1 if x>=y else 0)
df['hashtags'] = df['hashtags'].str.lower()

In [8]:
df['retweet']=df['retweet'].apply(lambda x: 0 if x=='FALSE' else 1)

In [9]:
df['no.of_hashtags'] = df['hashtags'].str.split(',').apply(len)


In [10]:
df['no.of_mentions'] = df['mentions'].str.split(',').apply(len)

In [11]:
df['no.of_words'] = df['tweet'].str.split(' ').apply(len)

In [12]:
df['no.of_photos'] = df['photos'].str.split('jpg').apply(len)-1

In [13]:
user_tweet_counts = df.groupby('user_id').size().reset_index(name='tweet_per_user')
df = df.merge(user_tweet_counts, on='user_id', how='left')

In [14]:
df.head(20)

,Unnamed: 0,id,conversation_id,created_at,date,time,time_zone,user_id,username,name,...,label,label_new,pop_score_log,class,category,no.of_hashtags,no.of_mentions,no.of_words,no.of_photos,tweet_per_user
0,0,1.511390e+18,1.511390e+18,2022-04-05 17:05:33,2022-04-05,17:05:33,UTC,9.659076e+08,ganeshhirennava,Ganesh H,...,2.0,2.0,0.693147,0.076634,0,5,1,12,0,1
1,1,1.509950e+18,1.509950e+18,2022-04-01 17:45:50,2022-04-01,17:45:50,UTC,1.395262e+09,LaxmiputraDesai,DESAI LAXMIPUTRA,...,3.0,4.0,3.737670,0.413236,1,1,1,16,0,1
2,2,1.509589e+18,1.509589e+18,2022-03-31 17:52:11,2022-03-31,17:52:11,UTC,1.468286e+18,hops_neil,Wilson Morgan ⚡⚡⚡⚡🌩️🌩️🌩️,...,1.0,2.0,0.000000,0.000000,0,1,1,12,1,1
3,3,1.510501e+18,1.510501e+18,2022-04-03 06:12:39,2022-04-03,06:12:39,UTC,1.328367e+18,repalli_siva,#Hyderabadi,...,2.0,3.0,1.386294,0.153268,1,1,1,16,0,1
4,4,1.513160e+18,1.513160e+18,2022-04-10 14:20:31,2022-04-10,14:20:31,UTC,1.489971e+18,PrashanthSkp1,Prashanth Skp,...,3.0,3.0,1.791759,0.198097,1,2,2,24,0,1
5,5,1.510474e+18,1.510474e+18,2022-04-03 04:26:54,2022-04-03,04:26:54,UTC,1.208072e+18,PremD8055,ಡಿ ಬಾಸ್ ಸೆಲೆಬ್ರಿಟಿಸ್ - ಆನೇಕಲ್,...,3.0,4.0,2.708050,0.299402,1,3,2,33,0,1
6,6,1.512626e+18,1.512626e+18,2022-04-09 02:59:03,2022-04-09,02:59:03,UTC,1.219989e+18,Kumar_ind90,Kumar_Nfan,...,2.0,3.0,1.386294,0.153268,1,2,1,20,0,1
7,7,1.510867e+18,1.510867e+18,2022-04-04 06:27:53,2022-04-04,06:27:53,UTC,1.457588e+18,ImTanjin93,তানজিন তিশা (উর্মি) 🇧🇩🇧🇩,...,3.0,3.0,2.079442,0.229903,1,3,1,52,1,29
8,8,1.510854e+18,1.510854e+18,2022-04-04 05:35:25,2022-04-04,05:35:25,UTC,1.457588e+18,ImTanjin93,তানজিন তিশা (উর্মি) 🇧🇩🇧🇩,...,2.0,2.0,0.693147,0.076634,0,5,1,19,1,29
9,9,1.510851e+18,1.510851e+18,2022-04-04 05:25:17,2022-04-04,05:25:17,UTC,1.457588e+18,ImTanjin93,তানজিন তিশা (উর্মি) 🇧🇩🇧🇩,...,1.0,2.0,0.000000,0.000000,0,6,1,15,1,29


In [15]:
df.columns

Index(['Unnamed: 0', 'id', 'conversation_id', 'created_at', 'date', 'time',
       'time_zone', 'user_id', 'username', 'name', 'place', 'tweet',
       'mentions', 'urls', 'photos', 'replies_count', 'retweet_count',
       'likes_count', 'hashtags', 'link', 'retweet', 'geo', 'language',
       'pop_score', 'label', 'label_new', 'pop_score_log', 'class', 'category',
       'no.of_hashtags', 'no.of_mentions', 'no.of_words', 'no.of_photos',
       'tweet_per_user'],
      dtype='object')

In [16]:
dt,dx=train_test_split(df,test_size=0.2,random_state=18)

In [17]:
dt=dt.reset_index()

In [18]:
dx=dx.reset_index()

In [19]:
X_train=dt["tweet"].values

In [21]:
Y_train=np.asarray(dt['category'].values).astype('float32')

In [22]:
X_test=dx["tweet"].values

In [23]:
Y_test=np.asarray(dx['category'].values).astype('float32')

In [24]:
additional_features=dt[["retweet_count",'likes_count','replies_count','no.of_hashtags', 'no.of_mentions', 'no.of_words',
       'no.of_photos','tweet_per_user']].values

In [25]:
additional_features_t=dx[["retweet_count",'likes_count','replies_count','no.of_hashtags', 'no.of_mentions', 'no.of_words',
       'no.of_photos',"tweet_per_user"]].values

In [26]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.4 MB/s eta 0:00:00


In [27]:
from tensorflow.keras.layers import Input, Concatenate, Dense

In [29]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer, TFRobertaModel, RobertaTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [30]:
bertweet_model = TFBertModel.from_pretrained("vinai/bertweet-base")
bertweet_tokenizer = BertTokenizer.from_pretrained("vinai/bertweet-base")
roberta_model = TFRobertaModel.from_pretrained("roberta-base")
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some layers from the model checkpoint at vinai/bertweet-base were not used when initializing TFBertModel: ['lm_head', 'roberta']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['bert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertweetTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'roberta.embeddings.position_ids', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [31]:
num_numerical_features = 8 # Number of numerical features
num_sentiment_classes =3   # Number of sentiment classes (e.g., 3 for negative, neutral, positive)

In [32]:
input_tweet_text = tf.keras.layers.Input(shape=(20,), dtype=tf.int32, name="input_tweet_text")
input_numerical_features = tf.keras.layers.Input(shape=(num_numerical_features,), dtype=tf.float32, name="input_numerical_features")


In [33]:
input_ids = bertweet_tokenizer(input_tweet_text, padding=True, truncation=True, return_tensors="tf")["input_ids"]

ValueError: ignored

In [ ]:




# Define your custom layers for classification


# Define input layers

# Tokenize the tweet text using BERTweet tokenizer


# Pass the tokenized tweet text through BERTweet and RoBERTa models
bertweet_output = bertweet_model(input_ids)["last_hidden_state"]
roberta_output = roberta_model(input_ids)["last_hidden_state"]

# Use mean pooling to aggregate BERTweet and RoBERTa outputs
bertweet_mean = tf.reduce_mean(bertweet_output, axis=1)
roberta_mean = tf.reduce_mean(roberta_output, axis=1)

# Concatenate BERTweet and RoBERTa outputs with numerical features
concatenated_features = tf.keras.layers.concatenate([bertweet_mean, roberta_mean, input_numerical_features])

# Add custom classification layers
dense_layer = tf.keras.layers.Dense(512, activation="relu")(concatenated_features)
dropout_layer = tf.keras.layers.Dropout(0.1)(dense_layer)
output_layer = tf.keras.layers.Dense(num_sentiment_classes, activation="softmax")(dropout_layer)

# Create the model
model = tf.keras.Model(inputs=[input_tweet_text, input_numerical_features], outputs=output_layer)

# Compile the model with an appropriate loss function and optimizer
model.compile(
    loss="categorical_crossentropy",  # Choose an appropriate loss function
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-5),  # Use AdamW optimizer as mentioned
    metrics=["accuracy"]
)

# Split the dataset into training, validation, and test sets
X_train, X_val, X_test, y_train, y_val, y_test = train_test_split(...)

# Preprocess numerical features using Min-Max scaling
scaler = MinMaxScaler()
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_val[numerical_features] = scaler.transform(X_val[numerical_features])
X_test[numerical_features] = scaler.transform(X_test[numerical_features])

# Create TensorFlow Datasets or DataGenerators for training, validation, and test sets

# Train the model
model.fit(
    [X_train["tweet_text"], X_train[numerical_features]],
    y_train,
    validation_data=([X_val["tweet_text"], X_val[numerical_features]], y_val),
    epochs=...,  # Choose an appropriate number of epochs
    batch_size=32,
    callbacks=...  # Add any desired callbacks, e.g., early stopping
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate([X_test["tweet_text"], X_test[numerical_features]], y_test)

# Make predictions using the trained model
predictions = model.predict([X_test["tweet_text"], X_test[numerical_features]])

# Further post-processing and analysis as needed


In [ ]:
concatenated_features = Concatenate(axis=-1)([train_enc,additional_features])

In [ ]:
gsc_enc=GraphSageConv(channels=776)([concatenated_features,adjacency_matrix_sparse_train])

In [ ]:
from tensorflow import keras
model = keras.Sequential()
model.add(keras.layers.Dense(384, activation='relu', input_shape=(776,))) # 768 + 8
model.add(keras.layers.Dropout(0.2))  # Dropout layer with a dropout rate of 0.2
model.add(keras.layers.Dense(192, activation='relu'))
model.add(keras.layers.Dropout(0.2))  # Dropout layer with a dropout rate of 0.2
model.add(keras.layers.Dense(96, activation='relu'))
model.add(keras.layers.Dropout(0.2))  # Dropout layer with a dropout rate of 0.2
model.add(keras.layers.Dense(48, activation='relu'))
model.add(keras.layers.Dropout(0.2))  # Dropout layer with a dropout rate of 0.2
model.add(keras.layers.Dense(24, activation='relu'))
model.add(keras.layers.Dropout(0.2))  # Dropout layer with a dropout rate of 0.2
model.add(keras.layers.Dense(12, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()



In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(gsc_enc,Y_train, epochs=10, batch_size=Y_train.shape[0])


In [ ]:
test_enc=create_bert_encodings(X_test)

In [ ]:
concatenated_features_t = Concatenate(axis=-1)([test_enc,additional_features_t])

In [ ]:
gsc_enc_t=GraphSageConv(channels=776)([concatenated_features_t,adjacency_matrix_sparse_test])

In [ ]:
y_pred=model.predict(gsc_enc_t)

In [ ]:
y_pred

In [ ]:
y_pred=np.where(y_pred>y,1,0)

In [ ]:
precision_score(Y_test,y_pred)

In [ ]:
recall_score(Y_test,y_pred)

In [ ]:
f1_score(Y_test,y_pred)

In [ ]:
accuracy_score(Y_test,y_pred)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(Y_test,y_pred))